In [2]:
import requests
import os
from dotenv import load_dotenv
load_dotenv("sjoerdAzure.env")  # Load environment variables from .env file
import time

import typing

from sklearn.metrics import cohen_kappa_score, classification_report
import krippendorff
import yaml

import pandas as pd

import config
import src
import tqdm
import json
import numpy as np
import logging

#import cltrier_lib as lib
import pyreadr
import yaml
pd.set_option('display.max_colwidth', 100) 

In [3]:
#set up helper variables and functions:
CFG = config.Config()

def load_json(path: str):
    with open(path, encoding='utf-8') as fp:
        return json.load(fp)
    
#set option variables:

#set options to low temperature (0,1):
options_low_str = """
seed: 42
temperature: 0.1
"""

options_low = yaml.safe_load(options_low_str)

MODELsmall: str = 'llama3.1:8b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'
MODELlarge: str = 'llama3.1:70b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'
MODELgpt4o = "nf-gpt-4o-2024-08-06" # in principe is er nu van elk model een nf (no filter) en een normale versie beschikbaar, de no filter versies zijn alleen voor onderzoekers beschikbaar voor analyze van content die niet door de filter heen zou komen.
MODELgpt4T = "nf-gpt-4-turbo" # Can be gpt-35-turbo, gpt-4-turbo, gpt-4 or Meta-Llama-3-8B-Instruct.
MODELgpt4 = "nf-gpt-4" # Can be gpt-35-turbo, gpt-4-turbo, gpt-4 or Meta-Llama-3-8B-Instruct.

options_zero_str = """
seed: 42
temperature: 0
"""
options_zero = yaml.safe_load(options_zero_str)

temperature_0 : int = 0
SEED: int = 42
MAX10: int = 10
TOPP1: int = 1


options_large_str = """
seed: 42
temperature: 0
num_predict: 2000
"""
options_large = yaml.safe_load(options_large_str)

#load environment variables:
api_key = os.environ.get('sjoerd_key')

#setttings:
api_endpoint = "https://ai-research-proxy.azurewebsites.net/chat/completions"
api_endpoint_embed = "https://ai-research-proxy.azurewebsites.net/embeddings"
####### API REQUEST FORMATTING ######
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + api_key
}

In [4]:
# Read data from .rds file
tweets = pyreadr.read_r('data/TWON_German_data/GermanyMdBTweets_2023.rds')
replies = pyreadr.read_r('data/TWON_German_data/GermanyReplies2023.rds')
users = pyreadr.read_r('data/TWON_German_data/Germany2023Users.rds')


LibrdataError: Invalid file, or file has unsupported features

In [ ]:
#this is highly nested data...